## N-Gram 語言模型

N-Gram 語言模型是基於統計的語言模型算法，
基於馬可夫假設將文本中的內容取最靠近的 N 個字當作條件機率計算的先驗條件，
形成長度是 N 的字詞片段序列。每個字詞片段及稱為gram。

1. 了解如何實作與使用 N-gram 模型
2. 透過實作更加了解 N-gram 模型
3. 使用NLTK建構 N-gram 模型

### 了解如何實作與使用N_gram模型
N_gram模型是基於統計的基礎語言模型，在這次的課程中，我們會學習到如何使用pyhon來實作N_gram的語言模型。
課程中的範例我們會使用[傲慢與偏見的英文文本](https://www.gutenberg.org/ebooks/1342)來進行。

### 進行資料清洗
在搭建語言模型前，需要將文本資料根據需求進行清洗，像是去除標點符號、將文字正規化(大小寫轉換)等等

In [25]:
#讀取文本資料

import urllib, requests

books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']


url_template = f'https://www.gutenberg.org/cache/epub/{book}/pg{book}.txt'
response = requests.get(url_template)
txt = response.text

#檢查文本
print(len(txt), ',', txt[:50] , '...')

717572 , ﻿The Project Gutenberg EBook of Pride and Prejudic ...


在開始搭建N-gram模型之前，先對文本進行清洗
(移除非英文與數字字元，且將所有的字轉為小寫)

In [26]:
import re

words = re.split('[^A-Za-z]+', txt.lower())
words = [x for x in words if x != ''] # 移除空字串

print(len(words))

125897


### 建立詞頻字典
N-gram模型所需的機率需要使用詞頻來做計算，因此先對文本進行詞頻計算。
這裡使用Unigram與Bigram進行

In [27]:
# 建立詞頻字典unigram
unigram_frequecy = dict()

# 計算詞頻
for word in words:
    unigram_frequecy[word] = unigram_frequecy.get(word, 0) + 1
    # print(unigram_frequecy)
    # print("==============")
    # print(unigram_frequecy[word])
    # print("==============")
    # print(unigram_frequecy.get(word, 0) )
    # print("==============")

# 根據詞頻排序, 並轉換為(word, count)格式
unigram_frequecy = sorted(unigram_frequecy.items(), key=lambda word_count: word_count[1], reverse=True)

# 查看詞頻前10的字詞
print(unigram_frequecy[:10])

[('the', 4507), ('to', 4243), ('of', 3730), ('and', 3658), ('her', 2225), ('i', 2070), ('a', 2012), ('in', 1937), ('was', 1847), ('she', 1710)]


In [ ]:
# bigram
bigram_frequency = dict()

for i in range(0, len(words)-1):
    bigram_frequency[tuple(words[i:i+2])] = bigram_frequency.get(tuple(words[i:i+2]), 0) + 1
    
# 根據詞頻排序, 並轉換為(word, count)格式
bigram_frequency = sorted(bigram_frequency.items(), key=lambda words_count: words_count[1], reverse=True)

# 查看詞頻前10的字詞
bigram_frequency[:10]

### 搭建N-gram模型
接下來可以隨機選取一個初始的單字，再利用建立好的N-gram模型(這裡我們採用bigram)，
來預測機率最高的10個字組成的句子，若沒有找到接續的詞，則中斷預測。

這裡我們建立的Bigram詞頻表為(word_pairs, count)格式，其中word_pairs為(word_1, word_2)。

In [ ]:
# 建立 Bigram 模型
def do_bigram_prediction(bigram_freq, start_word, num_words):
    #定義起始字
    pred_words = [start_word]
    word = start_word
    for i in range(num_words):
        # 找尋下一個字
        word = pred_words[i]
        word = next((word_pairs[1] for (word_pairs, count) in bigram_freq if word_pairs[0] == word), None)
        
        if not word:
            break
        else:
            pred_words.append(word)
    return pred_words

In [ ]:
# 使用建立好的 Bigram 模型進行預測
import random

# 隨機選取起始字
start_word = random.choice(words)
print(f'初始字: {start_word}')

# 使用選取的起始字預測接下來的字詞(共10個字)
pred_words = do_bigram_prediction(bigram_frequency, start_word, 10)

# 顯示預測結果
print(f"預測句子: {' '.join(pred_words)}")

### 加入選取權重
依照上述的模型，會產生的問題是，如果開頭的字都相同(ex: of)，則後續的預測句子都一定會相同，
為了修正這樣的問題，我們可以利用N-gram詞頻中的頻率當作權重，增加隨機性，以確保後續預測的句子都不一定會相同。

In [ ]:
# 權重選取
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w > r:
         return c
      upto += w
    
def do_bigram_weighted_prediction(bigram_freq, start_word, num_words):
    pred_words = [start_word]
    # word = start_word
    for i in range(num_words):
        # 選取所有符合條件的2gram
        word = pred_words[i]
        words_candidates = [word_pairs_count for word_pairs_count in bigram_freq if word_pairs_count[0][0] == word]
        if not words_candidates:
            break
        else:
            #根據加權機率選取可能的字詞
            pred_words.append(weighted_choice(words_candidates)[1])
            
    return pred_words

In [ ]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_bigram_weighted_prediction(bigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

In [ ]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_bigram_weighted_prediction(bigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

可以發現，相同的起始字，也可能得到不同的預測結果。

接下來我們可以根據上述的搭建方式，來建立泛化的N-gram模型(可自由選定N值)，根據N值的不同可以得到如Unigram(N=1)，Bigram(N=2)，Trigram(N=3)等。

In [ ]:
def generateNgram(N):
    gram_frequency = dict()
    
    # 避免N值過大，導致記憶體崩潰問題(先設定N < 100)
    assert N > 0 and N < 100
    
    # 建立N-gram的頻率字典
    for i in range(len(words)-(N-1)):
        gram_frequency[tuple(words[i:i+N])] = gram_frequency.get(tuple(words[i:i+N]), 0) + 1

    # 根據詞頻排序, 並轉換為(word, count)格式
    gram_frequency = sorted(gram_frequency.items(), key=lambda words_count: words_count[1], reverse=True)
    
    return gram_frequency

In [ ]:
# 建立Trigram
trigram_frequency = generateNgram(3)

# 查看詞頻前10的字詞
trigram_frequency[:10]

In [ ]:
#建立N-gram預測function
def do_ngram_weighted_prediction(gram_freq, start_word, num_words):
    pred_words = [start_word]
    # word = start_word
    for i in range(num_words):
        # 選取所有符合條件
        word = pred_words[i]
        words_candidates = [word_pairs_count for word_pairs_count in gram_freq if word_pairs_count[0][0] == word]
        
        if not words_candidates:
            break
        else:
            #根據加權機率選取可能的字詞
            pred_words.append(weighted_choice(words_candidates)[1])
            
    return pred_words

In [ ]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_ngram_weighted_prediction(trigram_frequency, start_word, 10)
print(f"預測推薦字詞: {' '.join(pred_words)}")

### 使用NLTK套件搭建N-gram模型

NLTK(Natural Language Toolkit)使一個python的自然語言處理套件，內含許多應用，這裡我們會使用NLTK來建立N-gram模型。

首先需要進行套件安裝，只需要執行

```bash
pip install nltk
```

In [ ]:
import collections
from nltk import ngrams

#使用NLTK API搭建Bigram
bigram_frequency = ngrams(words, n=2)

#使用collectins套件計算詞頻
bigram_frequency = collections.Counter(bigram_frequency)


In [ ]:
import re
import collections
from nltk import ngrams

In [ ]:
#讀取文本資料

import urllib, requests

books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']


url_template = f'https://www.gutenberg.org/cache/epub/{book}/pg{book}.txt'
response = requests.get(url_template)
txt = response.text

#檢查文本
print(len(txt), ',', txt[:50] , '...')

In [ ]:
words = re.split('[^A-Za-z]+', txt.lower())
words = [x for x in words if x != ''] # 移除空字串

print(len(words))

In [ ]:
# 建立bigram詞頻 (注意這裡回傳的是generator)
bigram_frequency = ngrams(words, n=2) 

# 計算詞頻
bigram_frequency = collections.Counter(bigram_frequency)

In [ ]:
bigram_frequency